In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from lightgbm import LGBMClassifier
import lightgbm as lgbm
import optuna
import logging

e:\Python Workspace Aur\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Data

train_df = pd.read_csv("E:/Python Workspace Aur/Playground Kaggle/playground-series-s5e11/train.csv")
test_df = pd.read_csv("E:/Python Workspace Aur/Playground Kaggle/playground-series-s5e11/test.csv")

In [3]:
# Preprocess

X = train_df.drop(['id', 'loan_paid_back'], axis=1)
y = train_df['loan_paid_back']
X_test_asli = test_df.drop('id', axis=1) 

numerical_features = X.select_dtypes(include=np.number).columns
categorical_features = X.select_dtypes(include='object').columns

for col in categorical_features:
    X[col] = X[col].astype('category')
    X_test_asli[col] = X_test_asli[col].astype('category')

print("Tipe data kategorikal telah diubah.")

numerical_transformer = StandardScaler()

Tipe data kategorikal telah diubah.


In [4]:
# Data Split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [5]:
# Process

numerical_transformer.fit(X_train[numerical_features])

def preprocess_data(df, transformer, num_cols, cat_cols):
    scaled_num_data = transformer.transform(df[num_cols])
    
    num_df = pd.DataFrame(scaled_num_data, columns=num_cols, index=df.index)
    
    cat_df = df[cat_cols]
    
    return pd.concat([num_df, cat_df], axis=1)

X_train_processed = preprocess_data(X_train, numerical_transformer, numerical_features, categorical_features)
X_val_processed = preprocess_data(X_val, numerical_transformer, numerical_features, categorical_features)

print(f"Bentuk X_train: {X_train_processed.shape}")

Bentuk X_train: (475195, 11)


In [6]:
# Objection function for Optuna

def objective(trial):
    params = {
        'objective': 'binary',
        'metric': 'auc',
        'n_estimators': 2000, 
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.05),
        'num_leaves': trial.suggest_int('num_leaves', 20, 80),
        'max_depth': trial.suggest_int('max_depth', 5, 12),
        'min_child_samples': trial.suggest_int('min_child_samples', 20, 100),
        'subsample': trial.suggest_float('subsample', 0.7, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.7, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 0.5),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 0.5),
        'n_jobs': -1,
        'random_state': 42
    }
    
    model = LGBMClassifier(**params)
    
    model.fit(
        X_train_processed, 
        y_train,
        eval_set=[(X_val_processed, y_val)],
        eval_metric='auc',
        categorical_feature=list(categorical_features),
        callbacks=[lgbm.early_stopping(100, verbose=False),
                   optuna.integration.LightGBMPruningCallback(trial, 'auc')]
    )
    
    trial.set_user_attr('best_iteration', model.best_iteration_)
    
    preds = model.predict_proba(X_val_processed)[:, 1]
    auc_score = roc_auc_score(y_val, preds)
    
    return auc_score

In [7]:
optuna.logging.set_verbosity(optuna.logging.WARNING)
study = optuna.create_study(direction='maximize')

study.optimize(objective, n_trials=100) 

print(f"Skor AUC validasi terbaik: {study.best_value:.6f}")
print("Parameter terbaik:")
print(study.best_params)

[LightGBM] [Info] Number of positive: 379595, number of negative: 95600
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004848 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1333
[LightGBM] [Info] Number of data points in the train set: 475195, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.798819 -> initscore=1.378932
[LightGBM] [Info] Start training from score 1.378932
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gai

In [8]:
best_params = study.best_params
best_iteration = study.best_trial.user_attrs['best_iteration']

X_full_processed = preprocess_data(X, numerical_transformer, numerical_features, categorical_features)
X_test_processed = preprocess_data(X_test_asli, numerical_transformer, numerical_features, categorical_features)

In [9]:
# Model Training 

final_model = LGBMClassifier(
    objective='binary',
    metric='auc',
    **best_params,
    n_estimators=best_iteration,
    n_jobs=-1,
    random_state=42
)

final_model.fit(
    X_full_processed, 
    y,
    categorical_feature=list(categorical_features)
)

[LightGBM] [Info] Number of positive: 474494, number of negative: 119500
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003339 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1333
[LightGBM] [Info] Number of data points in the train set: 593994, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.798820 -> initscore=1.378933
[LightGBM] [Info] Start training from score 1.378933


,boosting_type,'gbdt'
,num_leaves,37
,max_depth,10
,learning_rate,0.033904357777413865
,n_estimators,1796
,subsample_for_bin,200000
,objective,'binary'
,class_weight,None
,min_split_gain,0.0
,min_child_weight,0.001
,min_child_samples,60


In [10]:
# Model Training
test_pred_proba_lgbm = final_model.predict_proba(X_test_processed)[:, 1]

submission_df_lgbm = pd.DataFrame({
    'id': test_df['id'],
    'loan_paid_back': test_pred_proba_lgbm
})

submission_filename = 'submission8-lgbm_native_optuna.csv'
submission_df_lgbm.to_csv(submission_filename, index=False)

print(f"\nBerhasil! File submission baru telah dibuat: {submission_filename}")
print(submission_df_lgbm.head())


Berhasil! File submission baru telah dibuat: submission8-lgbm_native_optuna.csv
       id  loan_paid_back
0  593994        0.944457
1  593995        0.979524
2  593996        0.586833
3  593997        0.918730
4  593998        0.967419
